In [2]:
!pip install tensorflow==2.5.0

     |████████████████████████████████| 454.3 MB 16 kB/s 
     |████████████████████████████████| 462 kB 39.9 MB/s 
     |████████████████████████████████| 1.2 MB 43.4 MB/s 
     |████████████████████████████████| 4.0 MB 45.7 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.39.0
    Uninstalling grpcio-1.39.0:
      Successfully uninstalled grpcio-1.39.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib as mpl
import matplotlib.pyplot as plt

np.random.seed(0)
tf.random.set_seed(0)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# TPU 연결하고 싶을 때 import 
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

KeyError: ignored

#### 파일 불러오기

In [5]:
train_image = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/trainX_1000_norm_2.npy")
train_label = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/trainY_1000_2.npy")
train_label_num = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/trainY_num_1000_2.npy")

In [6]:
test_image = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/testX_1000_norm_2.npy")
test_label = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/testY_1000_2.npy")
test_label_num = np.load("/content/drive/MyDrive/ColabNotebooks/dataset/testY_num_1000_2.npy")

In [7]:
print(train_image.shape, test_image.shape)
print(train_label.shape, test_label.shape)
print(train_label_num.shape, test_label_num.shape)

(5000, 128, 128, 3) (4539, 128, 128, 3)
(5000,) (4539,)
(5000,) (4539,)


#### CNN 설계

In [8]:
LEARNING_RATE = 0.001
# N_EPOCHS = 100
N_EPOCHS = 30  # 수정
N_BATCHS = 100
DROPOUT_RATE = 0

N_TRAIN = train_image.shape[0]
N_TEST= test_image.shape[0]

In [9]:
if tf.device("/gpu:0"):  # 현재 장비 GPU로 설정되어 있는지 확인  
  print("True")

True


In [10]:
# label을 one-hot-encoding (매개변수로 int형 변수 들어가야 함)

y_train = keras.utils.to_categorical(train_label_num)
y_test = keras.utils.to_categorical(test_label_num)

In [11]:
print(y_train.shape, y_test.shape)

(5000, 5) (4539, 5)


In [12]:
train_dataset= tf.data.Dataset.from_tensor_slices((train_image, y_train)).shuffle(N_TRAIN).batch(N_BATCHS)

test_dataset= tf.data.Dataset.from_tensor_slices((test_image, y_test)).batch(N_BATCHS)

In [16]:
def test_model(): # dropout 생략
  model = keras.Sequential()

  model.add(layers.Dropout(DROPOUT_RATE))
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3), padding="same", strides=1))  
  model.add(layers.MaxPooling2D((2, 2)))
  
  model.add(layers.Dropout(DROPOUT_RATE))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same", strides = 1))
  
  model.add(layers.Dropout(DROPOUT_RATE))
  model.add(layers.Conv2D(128, (3, 3), activation='relu', padding="same", strides = 1))
  model.add(layers.MaxPooling2D((2, 2)))
  
  model.add(layers.Dropout(DROPOUT_RATE))
  model.add(layers.Conv2D(256, (3, 3), activation='relu', padding="same", strides = 1))
  model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Flatten())
  model.add(layers.Dropout(DROPOUT_RATE))
  model.add(layers.Dense(64, activation='relu'))

  model.add(layers.Dropout(DROPOUT_RATE))
  model.add(layers.Dense(units=5, activation="softmax"))

  return model

In [17]:
model = test_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [18]:
hist = model.fit(train_dataset, epochs=N_EPOCHS)

Epoch 1/30
50/50 [==============================] - 463s 9s/step - loss: 1.4473 - accuracy: 0.3246
Epoch 2/30
50/50 [==============================] - 455s 9s/step - loss: 1.3380 - accuracy: 0.3748
Epoch 3/30
50/50 [==============================] - 464s 9s/step - loss: 1.2644 - accuracy: 0.4126
Epoch 4/30
50/50 [==============================] - 469s 9s/step - loss: 1.2060 - accuracy: 0.4538
Epoch 5/30
50/50 [==============================] - 464s 9s/step - loss: 1.1241 - accuracy: 0.5110
Epoch 6/30
50/50 [==============================] - 466s 9s/step - loss: 1.0123 - accuracy: 0.5720
Epoch 7/30
50/50 [==============================] - 459s 9s/step - loss: 0.8642 - accuracy: 0.6510
Epoch 8/30
50/50 [==============================] - 463s 9s/step - loss: 0.6668 - accuracy: 0.7442
Epoch 9/30
50/50 [==============================] - 460s 9s/step - loss: 0.4753 - accuracy: 0.8200
Epoch 10/30
50/50 [==============================] - 462s 9s/step - loss: 0.2807 - accuracy: 0.9006
Epoch 11/

In [19]:
# 테스트 데이터 정확도 계산

loss, acc = model.evaluate(test_dataset)

46/46 [==============================] - 121s 3s/step - loss: 6.4866 - accuracy: 0.4724
